# Text Similarity and Clustering
Today we will talk about text similarity and clustering (chapter 6 of the textbook for this class).

In [1]:
terms = ['believe', 'beleive', 'bargain', 'elephant']

In [2]:
import numpy as np

def vectorize_terms(terms):
    """Calculates character vector for terms."""
    # Converts terms to lower case.
    terms = [term.lower() for term in terms]
    # Creates array with with every term as a list of its characters.
    terms = [np.array(list(term)) for term in terms]
    # Converts character to number, corresponding to unicode point.
    terms = [np.array([ord(char) for char in term]) for term in terms]
    return terms

In [3]:
vec_terms = vectorize_terms(terms)
vec_terms

[array([ 98, 101, 108, 105, 101, 118, 101]),
 array([ 98, 101, 108, 101, 105, 118, 101]),
 array([ 98,  97, 114, 103,  97, 105, 110]),
 array([101, 108, 101, 112, 104,  97, 110, 116])]

In [4]:
from scipy.stats import itemfreq

def create_boc_vectors(term_list):
    """Creates bag of character vectors for term list."""
    # Converts terms to lower case.    
    term_list = [term.lower() for term in term_list]
    # Gets unique characters from terms.
    unique_chars = np.unique(np.hstack([list(term) for term in term_list]))
    # Calcualtes character frequency for each term.
    term_counts = [{char: count for char, count in itemfreq(list(term))}
                  for term in term_list]
    # Appends vectors.
    boc_vectors = [np.array([int(term_count.get(char, 0)) for char in unique_chars])
                  for term_count in term_counts]
    return list(unique_chars), boc_vectors

In [6]:
unique_chars, boc_matrix = create_boc_vectors(terms)

In [7]:
import pandas

def display_boc_vectors(terms, unique_chars, boc_vectors):
    """Returns data frame with terms, unique_chars and boc_vectors"""
    return pandas.DataFrame(boc_vectors, index=terms, columns=unique_chars)

In [9]:
display_boc_vectors(terms, *create_boc_vectors(terms))

,a,b,e,g,h,i,l,n,p,r,t,v
believe,0,1,3,0,0,1,1,0,0,0,0,1
beleive,0,1,3,0,0,1,1,0,0,0,0,1
bargain,2,1,0,1,0,1,0,1,0,1,0,0
elephant,1,0,2,0,1,0,1,1,1,0,1,0


### Hamming Distance

In [11]:
def hamming_distance(vec_a, vec_b, normalize=False):
    if vec_a.shape != vec_b.shape:
        return None
    if not normalize:
        return (vec_a != vec_b).sum()
    else:
        return (vec_a != vec_b).mean()

In [12]:
hamming_distance(vec_terms[0], vec_terms[1])

2

In [13]:
def calculate_similarity_matrix(items, similarity_metric, **kwargs):
    similarity_matrix = np.empty((len(items), len(items)))
    similarity_matrix[:] = np.nan
    
    log = []
    for r_index, r_val in enumerate(items):
        for c_index, c_val in enumerate(items):
                if (c_index, r_index) not in log:
                    similarity_matrix[r_index][c_index] = similarity_metric(r_val, c_val, **kwargs)
                    log.append((r_index, c_index))
    
    return similarity_matrix

In [14]:
s_matrix = calculate_similarity_matrix(vec_terms, hamming_distance)
s_matrix

array([[ 0.,  2.,  6., nan],
       [nan,  0.,  6., nan],
       [nan, nan,  0., nan],
       [nan, nan, nan,  0.]])

In [15]:
def display_similarity_matrix(similarity_matrix, terms):
    return pandas.DataFrame(similarity_matrix, index=terms, columns=terms).T

In [16]:
display_similarity_matrix(s_matrix, terms)

,believe,beleive,bargain,elephant
believe,0.0,NaN,NaN,NaN
beleive,2.0,0.0,NaN,NaN
bargain,6.0,6.0,0.0,NaN
elephant,NaN,NaN,NaN,0.0


In [20]:
s_matrix = calculate_similarity_matrix(vec_terms, hamming_distance, normalize=True)
display_similarity_matrix(s_matrix, terms)

,believe,beleive,bargain,elephant
believe,0.000000,NaN,NaN,NaN
beleive,0.285714,0.000000,NaN,NaN
bargain,0.857143,0.857143,0.0,NaN
elephant,NaN,NaN,NaN,0.0


### Manhattan Distance

In [29]:
def manhattan_distance(vec_a, vec_b, normalize=False):
    if vec_a.shape != vec_b.shape:
        return None
    if not normalize:
        return abs(vec_a - vec_b).sum()
    else:
        return abs(vec_a - vec_b).mean()

In [36]:
s_matrix = calculate_similarity_matrix(vec_terms, manhattan_distance)
display_similarity_matrix(s_matrix, terms)

,believe,beleive,bargain,elephant
believe,0.0,NaN,NaN,NaN
beleive,8.0,0.0,NaN,NaN
bargain,38.0,42.0,0.0,NaN
elephant,NaN,NaN,NaN,0.0


In [37]:
s_matrix = calculate_similarity_matrix(vec_terms, manhattan_distance, normalize=True)
display_similarity_matrix(s_matrix, terms)

,believe,beleive,bargain,elephant
believe,0.000000,NaN,NaN,NaN
beleive,1.142857,0.0,NaN,NaN
bargain,5.428571,6.0,0.0,NaN
elephant,NaN,NaN,NaN,0.0


### Euclidean Distance

In [38]:
def euclidean_distance(vec_a, vec_b):
    if vec_a.shape != vec_b.shape:
        return None
    return np.sqrt(np.sum(np.square(vec_a - vec_b)))

In [39]:
s_matrix = calculate_similarity_matrix(vec_terms, euclidean_distance)
display_similarity_matrix(s_matrix, terms)

,believe,beleive,bargain,elephant
believe,0.000000,NaN,NaN,NaN
beleive,5.656854,0.000000,NaN,NaN
bargain,17.944358,19.235384,0.0,NaN
elephant,NaN,NaN,NaN,0.0


### Levenshtein Edit Distance

In [44]:
import copy

def levenshtein_edit_distance(vec_a, vec_b):
    if vec_a == vec_b: return 0
    if len(vec_a) == 0: return(len(vec_b))
    if len(vec_b) == 0: return(len(vec_a))
    # Initialize edit distance matrix.
    edit_matrix = []
    # Initialize two distance matrices.
    d_a = [0] * (len(vec_b) + 1)
    d_b = [0] * (len(vec_a) + 1)
    # d_a: the previous row fo edit distances.
    for i in range(len(d_a)): d_a[i] = i
    # d_b: the current row of edit distances.
    for i in range(len(vec_a)):
        d_b[0] = i + 1
        # Compute cost as per algorithm.
        for j in range(len(vec_b)):
            cost = 0 if vec_a[i] == vec_b[j] else 1
            d_b[j + 1] = min(d_b[j] + 1, d_a[j + 1] + 1, d_a[j] + cost)
        # Assign d_b to d_a for next iteration.
        for j in range(len(d_a)):
            d_a[j] = d_b[j]
        # Copy d_b to the edit matrix.
        edit_matrix.append(copy.copy(d_b))
    # Compute the final edit distance and edit matrix.
    distance = d_b[len(vec_b)]
    edit_matrix = np.array(edit_matrix)
    edit_matrix = edit_matrix.T
    edit_matrix = edit_matrix[1:,]
    print(edit_matrix)
    edit_matrix = pandas.DataFrame(data=edit_matrix,
                                  index=list(vec_b),
                                  columns=list(vec_a))
    return distance, edit_matrix    

In [45]:
levenshtein_edit_distance('banana', 'apple')

[[1 1 2 3 4 5]
 [2 2 2 3 4 5]
 [3 3 3 3 4 5]
 [4 4 4 4 4 5]
 [5 5 5 5 5 5]
 [0 0 0 0 0 0]]


ValueError: Shape of passed values is (6, 6), indices imply (6, 5)